# Sentiment Anlysis
- In this file we will perform sentiment analysis but the use of `LLM`

# Import Packages

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import torch
import transformers
import datasets
from datasets import load_dataset

# Load Dataset

In [3]:
# Download the dataset
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0
 50% 1.00M/1.99M [00:00<00:00, 2.01MB/s]
100% 1.99M/1.99M [00:00<00:00, 3.38MB/s]


In [4]:
# Unzip the dataset
!unzip twitter-entity-sentiment-analysis.zip

Archive:  twitter-entity-sentiment-analysis.zip
  inflating: twitter_training.csv    
  inflating: twitter_validation.csv  


In [5]:
df = pd.read_csv("/content/twitter_training.csv")
df.head(2)

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...


# Observation
- First we can add a columns name.
- Second we can remove unecessary columns.

In [6]:
df.columns = ["id", "entity", "sentiment", "content"]
df.head(2)

,id,entity,sentiment,content
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...


In [7]:
df.drop(columns=["id", "entity"], inplace=True)
df.head(2)

,sentiment,content
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...


# Step we can do
- first we can remove null and duplicates valeus.
- second we can convert the data into hugging face datasets so that its easy for me apply preprocessing.
- third we can do tokenization.
- fourth we can train the model.
- we can also encode the sentiment labels

In [8]:
df.isnull().sum()

,0
sentiment,0
content,686


In [9]:
df.dropna(inplace=True)

In [10]:
df.isnull().sum()

,0
sentiment,0
content,0


In [11]:
df.duplicated().sum()

4227

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.duplicated().sum()

0

In [14]:
df.shape

(69768, 2)

In [15]:
df=df.sample(10000)

In [16]:
df.reset_index(drop=True, inplace=True)

In [17]:
df['sentiment'].unique()

array(['Irrelevant', 'Negative', 'Positive', 'Neutral'], dtype=object)

In [18]:
df['sentiment'] = df['sentiment'].map({
    "Positive": 0,  # Changed to 0
    "Negative": 1,  # Changed to 1
    "Neutral": 2,   # Changed to 2
    'Irrelevant': 3 # Changed to 3
})

In [19]:
df.head(2)

,sentiment,content
0,3,Wow. Netflix is taking the right steps.
1,3,"""and so it this a symptom of something bigger ..."


# Convert the data into hugging face dataset

In [20]:
data=datasets.Dataset.from_pandas(df)
data

Dataset({
    features: ['sentiment', 'content'],
    num_rows: 10000
})

# split the data into train and test set

In [21]:
data=data.train_test_split(test_size=0.2)

In [22]:
data

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'content'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentiment', 'content'],
        num_rows: 2000
    })
})

In [23]:
# train data
train_data=data["train"]


# test data
test_data=data["test"]

train_data.shape, test_data.shape

((8000, 2), (2000, 2))

# Load Tokenizer and Model

In [51]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=4)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Test the tokenizer

In [25]:
train_data[0]["content"]

'This baby looks dope!!'

In [26]:
# Encode the text
encode_embedding = tokenizer(train_data[0]["content"])

In [27]:
encode_embedding

{'input_ids': [101, 2023, 3336, 3504, 2079, 5051, 999, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [28]:
# Decode the embedding
tokenizer.decode(encode_embedding['input_ids'])

'[CLS] this baby looks dope!! [SEP]'

# Apply tokenization
- For this step we can make a custom dataset class for data loading

In [29]:
from torch.utils.data import Dataset, DataLoader
from torch import nn

In [30]:
class CutomData(Dataset):
    def __init__(
            self,data,tokenizer
    ):
        self.data = data
        self.content = data['content']
        self.sentiment = data['sentiment']
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        # get the content and sentiment of the specific index
        content = self.content[idx]
        sentiment = self.sentiment[idx]

        # apply tokenization
        content_embedding = self.tokenizer(content, padding = "max_length",truncation = True, return_tensors = "pt",max_length = 300)

        return {
            'input_ids' : content_embedding['input_ids'],
            'attention_mask' : content_embedding['attention_mask'],
            'labels' : torch.tensor(sentiment , dtype = torch.long)
        }

In [31]:
# Load the tokenize data
# Train data
tokenize_train_data = CutomData(train_data, tokenizer)

# Test data
tokenize_test_data = CutomData(test_data, tokenizer)

In [34]:
# Make a dataloader

# Train Loader
train_loader = DataLoader(tokenize_train_data, batch_size = 32, shuffle = True)

# Test Loader
test_loader = DataLoader(tokenize_test_data, batch_size = 32, shuffle = True)

In [35]:
# Check the data batch
for batch in train_loader:
    print(batch['input_ids'].shape, batch['attention_mask'].shape, batch['labels'].shape)
    break

torch.Size([32, 1, 300]) torch.Size([32, 1, 300]) torch.Size([32])


# Observation
- We can make a dataset that we can use for model training.
- we can train the model for a single batch so that we can see if all the training should be fine.
- if all are fine we can train the model on full dataset.

# View the model

In [52]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


#  Change the model out_features

In [53]:
# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

In [54]:
print("Model: ",model.classifier)
# Model in_features
in_features = model.classifier.in_features
print("in_features: ",in_features)

# Model Out_features
out_featurs = model.classifier.out_features
print("out_featurs: ",out_featurs)

Model:  Linear(in_features=768, out_features=4, bias=True)
in_features:  768
out_featurs:  4


In [55]:
# Update the lm_head for a 4-class classification task
# model.classifier = nn.Linear(model.classifier.in_features, 4)

In [56]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [57]:
# Unfreeze only the updated layer
for param in model.classifier.parameters():
    param.requires_grad = True

# Observation
- Now we can update the final classification layer.
- Now we can train the model

# set the optimizer and loss fun

In [58]:
# optimzer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Loss
loss_fn = torch.nn.CrossEntropyLoss()

In [59]:
optimizer, loss_fn

(AdamW (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 5e-05
     maximize: False
     weight_decay: 0.01
 ),
 CrossEntropyLoss())

# Make a compute fun
- In compute fun will calculate the accuracy, precession, recall and f1 score

In [60]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [61]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(predictions, labels, average='weighted'):
    """
    Compute accuracy, precision, recall, and F1 score.

    Args:
        predictions (list or np.array): Predicted labels.
        labels (list or np.array): True labels.
        average (str): Type of averaging for multi-class metrics.
                       Options: 'micro', 'macro', 'weighted'. Default is 'macro'.

    Returns:
        dict: A dictionary with accuracy, precision, recall, and F1 score.
    """
    # Accuracy
    accuracy = accuracy_score(labels, predictions)

    # Precision, Recall, F1 Score
    precision = precision_score(labels, predictions, average=average, zero_division=0)
    recall = recall_score(labels, predictions, average=average, zero_division=0)
    f1 = f1_score(labels, predictions, average=average, zero_division=0)

    # Return results as a dictionary
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

# Train the model in a single batch

In [62]:
from tqdm import tqdm

In [63]:
epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [64]:
from tqdm import tqdm

# Move the model to the correct device
model.to(device)

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()  # Ensure the model is in training mode

    # Training loop
    for batch in tqdm(train_loader, desc="Training"):
        # Move inputs and labels to the correct device
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)
        labels = batch['labels'].to(device)

        # Check shapes
        # print(f"Input IDs shape: {input_ids.shape}")
        # print(f"Attention mask shape: {attention_mask.shape}")
        # print(f"Labels shape: {labels.shape}")

        # Zero the gradients from the previous step
        optimizer.zero_grad()

        # Forward pass through the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        # Get the loss value
        loss = outputs.loss

        # Backpropagate the loss
        loss.backward()

        # # Update model parameters
        optimizer.step()
        break

    # Evaluation at the end of the epoch
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].squeeze(1).to(device)
            attention_mask = batch['attention_mask'].squeeze(1).to(device)
            labels = batch['labels'].to(device)


            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            break
    # Calculate metrics
    results = compute_metrics(all_preds, all_labels, average='weighted')
    print(f"Epoch {epoch + 1} Metrics: {results}")
    print(f"Loss: {loss.item()}")

Epoch 1/50


Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]


Epoch 1 Metrics: {'accuracy': 0.40625, 'precision': 0.1650390625, 'recall': 0.40625, 'f1_score': 0.2347222222222222}
Loss: 1.4037305116653442
Epoch 2/50


Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]


Epoch 2 Metrics: {'accuracy': 0.21875, 'precision': 0.0478515625, 'recall': 0.21875, 'f1_score': 0.07852564102564102}
Loss: 1.343056559562683
Epoch 3/50


...

Epoch 48 Metrics: {'accuracy': 0.21875, 'precision': 0.0478515625, 'recall': 0.21875, 'f1_score': 0.07852564102564102}
Loss: 1.3563835620880127
Epoch 49/50


Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]


Epoch 49 Metrics: {'accuracy': 0.21875, 'precision': 0.0478515625, 'recall': 0.21875, 'f1_score': 0.07852564102564102}
Loss: 1.3519545793533325
Epoch 50/50


Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 50 Metrics: {'accuracy': 0.34375, 'precision': 0.1181640625, 'recall': 0.34375, 'f1_score': 0.17587209302325582}
Loss: 1.3535239696502686


In [57]:
# print(model.config.num_labels)

2


# Observation
- we can train the model in a single batch its working.
- Now we can train the model in full dataset.

# Full Training

# implement a early stoping
- we can make a early stoping so that we can reduce the overfitting.

In [65]:
import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=5, delta=0, path="checkpoint.pt", verbose=False):
        """
        Args:
            patience (int): How many epochs to wait after the last improvement.
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
            path (str): Path to save the best model.
            verbose (bool): If True, prints a message for each improvement.
        """
        self.patience = patience
        self.delta = delta
        self.path = path
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss  # Use negative because lower loss is better

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Save the model when validation loss decreases."""
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


In [66]:
from tqdm import tqdm
import torch.nn as nn

# Early Stopping Class
early_stopping = EarlyStopping(patience=3, verbose=True, path="best_model.pt")

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    training_loss = 0.0

    # Training Loop
    for batch in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        training_loss += loss.item()

        loss.backward()
        optimizer.step()

    # Evaluation Loop
    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].squeeze(1).to(device)
            attention_mask = batch['attention_mask'].squeeze(1).to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(test_loader)
    training_loss /= len(train_loader)

    # Calculate Metrics
    results = compute_metrics(all_preds, all_labels, average="weighted")
    print(f"Epoch {epoch + 1} Metrics: {results}")
    print(f"Training Loss: {training_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Check for Early Stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered. Stopping training.")
        break

# Load the best model
model.load_state_dict(torch.load("best_model.pt"))

Epoch 1/10


Evaluating: 100%|██████████| 63/63 [00:16<00:00,  3.78it/s]


Epoch 1 Metrics: {'accuracy': 0.3255, 'precision': 0.20671920246002087, 'recall': 0.3255, 'f1_score': 0.21731111273087303}
Training Loss: 1.3645, Validation Loss: 1.3626
Validation loss decreased (inf --> 1.362560). Saving model...
Epoch 2/10


Evaluating: 100%|██████████| 63/63 [00:16<00:00,  3.80it/s]


Epoch 2 Metrics: {'accuracy': 0.309, 'precision': 0.24273850199774197, 'recall': 0.309, 'f1_score': 0.17096402515723272}
Training Loss: 1.3553, Validation Loss: 1.3566
Validation loss decreased (1.362560 --> 1.356551). Saving model...
Epoch 3/10


...

Epoch 9 Metrics: {'accuracy': 0.4425, 'precision': 0.40872352632695813, 'recall': 0.4425, 'f1_score': 0.38084709682345924}
Training Loss: 1.3194, Validation Loss: 1.3217
Validation loss decreased (1.325457 --> 1.321665). Saving model...
Epoch 10/10


Evaluating: 100%|██████████| 63/63 [00:16<00:00,  3.82it/s]


Epoch 10 Metrics: {'accuracy': 0.4625, 'precision': 0.40372184590654225, 'recall': 0.4625, 'f1_score': 0.404920884967721}
Training Loss: 1.3162, Validation Loss: 1.3161
Validation loss decreased (1.321665 --> 1.316060). Saving model...


<ipython-input-66-8fdd75fa10ac>:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))


<All keys matched successfully>

# Prediction
- Our model is train successfully now we can use the model for prediction.

In [69]:
def predict_sentiment(text):
    """Predicts the sentiment of a given text using the trained model."""

    # Tokenize the input text
    inputs = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt", max_length=300)

    # Move inputs to the correct device
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Make the prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()

    # Map the predicted class to the sentiment label
    sentiment_mapping = {
        0: "Positive",
        1: "Negative",
        2: "Neutral",
        3: "Irrelevant"
    }
    predicted_sentiment = sentiment_mapping.get(predicted_class, "Unknown")  # Handle cases where the class is not found
    return predicted_sentiment,predicted_class

# Testing

In [71]:
# Text Sample
text_to_predict = "This is not a  great product!"
predicted_sentiment = predict_sentiment(text_to_predict)
print(f"Predicted Sentiment for '{text_to_predict}': {predicted_sentiment}")

Predicted Sentiment for 'This is not a  great product!': ('Negative', 1)


In [73]:
# Text Sample
text_to_predict = "This service is terrible. I waited over an hour, and my issue still wasn't resolved"
predicted_sentiment = predict_sentiment(text_to_predict)
print(f"Predicted Sentiment for '{text_to_predict}': {predicted_sentiment}")

Predicted Sentiment for 'This service is terrible. I waited over an hour, and my issue still wasn't resolved': ('Negative', 1)


In [75]:
# Text Sample
text_to_predict = "'The new @CallofDuty for ps5 is 🔥🔥🔥🔥 Oh God 😭😍'"
predicted_sentiment = predict_sentiment(text_to_predict)
print(f"Predicted Sentiment for '{text_to_predict}': {predicted_sentiment}")

Predicted Sentiment for ''The new @CallofDuty for ps5 is 🔥🔥🔥🔥 Oh God 😭😍'': ('Negative', 1)


In [72]:
# Text Sample
text_to_predict = "The meeting is scheduled for 2 PM in the conference room"
predicted_sentiment = predict_sentiment(text_to_predict)
print(f"Predicted Sentiment for '{text_to_predict}': {predicted_sentiment}")

Predicted Sentiment for 'The meeting is scheduled for 2 PM in the conference room': ('Positive', 0)


In [74]:
# Text Sample
text_to_predict = "Call of duty warzone (livestream) w/ subs #Warzone youtu.be/7BhH_pjOMU4 via @YouTube Please come watch this AMAZING Call of Duty Warzone stream from this AMAZING streamer! It'd be really, really nice to give him some views and likes as well! 😀 #COD #CallofDuty #Warzone"
predicted_sentiment = predict_sentiment(text_to_predict)
print(f"Predicted Sentiment for '{text_to_predict}': {predicted_sentiment}")

Predicted Sentiment for 'Call of duty warzone (livestream) w/ subs #Warzone youtu.be/7BhH_pjOMU4 via @YouTube Please come watch this AMAZING Call of Duty Warzone stream from this AMAZING streamer! It'd be really, really nice to give him some views and likes as well! 😀 #COD #CallofDuty #Warzone': ('Neutral', 2)
